This notebook proposes a recipe to train Mistral 7B with IPO. I use the same datasets used by Hugging Face to train Zephyr.

More details in this article: [Fine-tune Better Chat Models with Distilled Identity Preference Optimization (IPO)](https://kaitchup.substack.com/p/fine-tune-better-chat-models-with)

There are two main sections in this notebook: The first one trains SFT and the second one trains IPO. Once SFT is done, I recommend to save your checkpoints somewhere and then to restart the runtime before training IPO.

First, we need all these dependencies:

*Note: As I write this notebook, TRL has to be install from source to use IPO.*

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/trl.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Import all the necessary packages.

In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments
)
from trl import SFTTrainer, DPOTrainer

# Distilled Supervised Fine-tuning

Load the tokenizer and configure padding

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Load and preprocess the version of ultrachat prepared by Hugging Face.
Since each row is a full dialog that can be very long, I only kept the first two turns to reduce the sequence length of the training examples.

In [ ]:
def format_ultrachat(ds):
  text = []
  for row in ds:
    if len(row['messages']) > 2:
      text.append("### Human: "+row['messages'][0]['content']+"### Assistant: "+row['messages'][1]['content']+"### Human: "+row['messages'][2]['content']+"### Assistant: "+row['messages'][3]['content'])
    else: #not all tialogues have more than one turn
      text.append("### Human: "+row['messages'][0]['content']+"### Assistant: "+row['messages'][1]['content'])
  ds = ds.add_column(name="text", column=text)
  return ds
dataset_train_sft = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")
dataset_test_sft = load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft[:5%]")

dataset_test_sft = format_ultrachat(dataset_test_sft)
dataset_train_sft = format_ultrachat(dataset_train_sft)


Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train_sft split:   0%|          | 0/207865 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/23110 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

Load the model that we will train with SFT and prepare it for QLoRA.

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}
)
model = prepare_model_for_kbit_training(model)
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False # Gradient checkpointing is used by default but not compatible with caching


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Define the configuration of LoRA.

In [ ]:
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.1,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj']
)

For this demonstration, I trained for only 300 steps. You should train for at least 3000 steps. One epoch would be ideal.

In [ ]:
training_arguments = TrainingArguments(
        output_dir="./results_mistral_sft/",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_steps=50,
        logging_steps=50,
        learning_rate=2e-5,
        eval_steps=50,
        max_steps=300,
        warmup_steps=30,
        lr_scheduler_type="linear",
)

Start training:

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_train_sft,
        eval_dataset=dataset_test_sft,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

Map:   0%|          | 0/207865 [00:00<?, ? examples/s]

Map:   0%|          | 0/1156 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
Currently training with a batch size of: 8
***** Running training *****
  Num examples = 207,865
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 300
  Number of trainable parameters = 13,631,488
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded 

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
50,1.311300,1.272258
100,1.248200,1.227211
150,1.197500,1.210177
200,1.229800,1.203750
250,1.220200,1.199879
300,1.179900,1.198481


***** Running Evaluation *****
  Num examples = 1156
  Batch size = 8
Saving model checkpoint to ./drive/MyDrive/results_mistral_sft/checkpoint-50
tokenizer config file saved in ./drive/MyDrive/results_mistral_sft/checkpoint-50/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/results_mistral_sft/checkpoint-50/special_tokens_map.json
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 1156
  Batch size = 8
Saving model checkpoint to ./drive/MyDrive/results_mistral_sft/checkpoint-100
tokenizer config file saved in ./drive/MyDri

TrainOutput(global_step=300, training_loss=1.2311581548055013, metrics={'train_runtime': 11820.9327, 'train_samples_per_second': 0.406, 'train_steps_per_second': 0.025, 'total_flos': 1.050519539810304e+17, 'train_loss': 1.2311581548055013, 'epoch': 0.02})

# Distilled IPO

Load and quantized Mistral 7B that will be trained with IPO.

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}
)
model = prepare_model_for_kbit_training(model)
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Load and quantized the reference model that have trained with SFT

In [ ]:

model_ref_name = "mistralai/Mistral-7B-v0.1"
bnb_config_ref = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model_ref = AutoModelForCausalLM.from_pretrained(
          model_ref_name, quantization_config=bnb_config, device_map={"": 0}
)
model_ref = PeftModel.from_pretrained(model_ref, "./results_mistral_sft/checkpoint-300/")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Format UltraFeedback for training DPO.

In [ ]:
def format_ultrafeedback(ds):
  text = dict()
  text['chosen'] = []
  text['rejected'] = []
  prompt = []
  for row in ds:
    prompt.append("### Human: "+row['prompt']+"### Assistant: ")
    for col in ['chosen','rejected']:
      text[col].append(row[col][1]['content'])
  ds = ds.rename_column("chosen", "chosen_json")
  ds = ds.rename_column("rejected", "rejected_json")
  ds = ds.rename_column("prompt", "prompt_text")

  ds = ds.add_column(name="chosen", column=text['chosen'])
  ds = ds.add_column(name="rejected", column=text['rejected'])
  ds = ds.add_column(name="prompt", column=prompt)
  return ds
dataset_train_dpo = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split="train_prefs")
dataset_test_dpo = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split="test_prefs[:5%]")

dataset_test_dpo = format_ultrafeedback(dataset_test_dpo)
dataset_train_dpo = format_ultrafeedback(dataset_train_dpo)

Define the configuration of LoRA

In [ ]:
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.1,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj']
)

For this demonstration, I trained for only 200 steps. IPO learns very slowly so you should train for at least 5000 steps. One epoch would be ideal.

In [ ]:
training_arguments = TrainingArguments(
        output_dir="./results_ipo/",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        per_device_eval_batch_size=2,
        log_level="debug",
        save_steps=50,
        logging_steps=50,
        learning_rate=1e-7,
        eval_steps=50,
        max_steps=200,
        warmup_steps=20,
        lr_scheduler_type="linear",
)

Start IPO training

In [ ]:
trainer = DPOTrainer(
    model,
    model_ref,
    args=training_arguments,
    beta=0.3,
    peft_config=peft_config,
    train_dataset=dataset_train_dpo,
    eval_dataset=dataset_test_dpo,
    tokenizer=tokenizer,
    loss_type='ipo'
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:258: UserWarning: When using DPODataCollatorWithPadding, you should set `max_length` in the DPOTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:265: UserWarning: When using DPODataCollatorWithPadding, you should set `max_prompt_length` in the DPOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:294: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
Currently training with a batch size of: 2
***** Running training *****
  Num examples = 

Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
50,99424.120000,125441.921875,97.705055,71.651131,0.640000,26.053938,-197.692398,-229.632278,-3.026671,-3.084124
100,100973.840000,125397.273438,97.662048,71.615837,0.640000,26.046211,-197.810043,-229.775681,-3.026341,-3.083869


***** Running Evaluation *****
  Num examples = 100
  Batch size = 2
Saving model checkpoint to ./drive/MyDrive/results_mistral_ipo_1e7b.3/checkpoint-50
tokenizer config file saved in ./drive/MyDrive/results_mistral_ipo_1e7b.3/checkpoint-50/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/results_mistral_ipo_1e7b.3/checkpoint-50/special_tokens_map.json
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 100
  Batch size = 2
Saving model checkpoint to ./drive/MyDrive/results_mistral_ipo_1e7b.3/checkpoint-100
tokenizer config f

Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
50,99424.120000,125441.921875,97.705055,71.651131,0.640000,26.053938,-197.692398,-229.632278,-3.026671,-3.084124
100,100973.840000,125397.273438,97.662048,71.615837,0.640000,26.046211,-197.810043,-229.775681,-3.026341,-3.083869
150,105010.980000,125371.101562,97.637917,71.597275,0.630000,26.040638,-197.871887,-229.856110,-3.026200,-3.083712
200,120630.510000,125361.476562,97.628128,71.589806,0.640000,26.038321,-197.896835,-229.888763,-3.026170,-3.083683


***** Running Evaluation *****
  Num examples = 100
  Batch size = 2
Saving model checkpoint to ./drive/MyDrive/results_mistral_ipo_1e7b.3/checkpoint-150
tokenizer config file saved in ./drive/MyDrive/results_mistral_ipo_1e7b.3/checkpoint-150/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/results_mistral_ipo_1e7b.3/checkpoint-150/special_tokens_map.json
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 100
  Batch size = 2
Saving model checkpoint to ./drive/MyDrive/results_mistral_ipo_1e7b.3/checkpoint-200
tokenizer confi

TrainOutput(global_step=200, training_loss=106509.8625, metrics={'train_runtime': 11196.128, 'train_samples_per_second': 0.286, 'train_steps_per_second': 0.018, 'total_flos': 0.0, 'train_loss': 106509.8625, 'epoch': 0.05})

Test the model with the following code:

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}
)

model = PeftModel.from_pretrained(model, "./results_ipo/checkpoint-200")
model.config.pad_token_id = tokenizer.pad_token_id
def generate(instruction):
    prompt = "### Human: "+instruction+"### Assistant: "
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
            input_ids=input_ids,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256
    )
    for seq in generation_output.sequences:
        output = tokenizer.decode(seq)
        print(output.split("### Assistant: ")[1].strip())
generate("Tell me about gravitation.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

1. Gravitation is the force of attraction between two objects. 2. Gravitation is a fundamental force in nature. 3. Gravitation is the force that keeps us on the ground. 4. Gravitation is the force that keeps the planets in orbit around the sun. 5. Gravitation is the force that keeps the moon in orbit around the earth. 6. Gravitation is the force that keeps the stars in orbit around the galaxy. 7. Gravitation is the force that keeps the galaxies in orbit around the universe. 8. Gravitation is the force that keeps the universe in orbit around the multiverse. 9. Gravitation is the force that keeps the multiverse in orbit around the omniverse. 10. Gravitation is the force that keeps the omniverse in orbit around the everything. 11. Gravitation is the force that keeps the everything in orbit around the nothing. 12. Gravitation is the force that keeps the nothing in orbit around the everything. 13. Gravitation is the force that keeps the everything in orbit around the nothing. 14. Gravitatio